# Qutrit Machine Learning Optimizaiton 

Author: Bora Basyildiz

## Imports

In [15]:
import torch
import numpy as np
from itertools import permutations
from itertools import product
import matplotlib.pyplot as plt
from math import sqrt

## Fidelity Calculation

In [111]:
def fidelity_Qutrit(J,B,M,target_gate,t,N_iter):
    #!/usr/bin/env python3
    # -*- coding: utf-8 -*-
    """
    Created on Mon Aug 16 4:33 2021

    @author: Bora & Alex
    """

    #Qutrit Gellmann Matricies, Pauli Matrix equivalents 
    sx = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]) 
    sy = np.array([[0,-1j, 0],[1j,0, 0], [0, 0, 0]]) 
    sz = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]) 
    l4 = np.array([[0,0,1],[0,0,0],[1,0,0]])
    l5 = np.array([[0,0,-1j],[0,0,0],[1j,0,0]])
    l6 = np.array([[0,0,0],[0,0,1],[0,1,0]])
    l7 = np.array([[0,0,0],[0,0,-1j],[0,1j,0]])
    l8 = 1/np.sqrt(3) * np.array([[1,0,0],[0,1,0],[0,0,-2]])
    id = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 0]])

    #Sigma +/- for example paper 
    raiseOne = np.array([[0,0,0],[1,0,0],[0,0,0]])
    raiseTwo = np.array([[0,0,0],[0,0,0],[0,1,0]])

    # Annhilation and Creation Operators
    annhilate = np.array([[0,1,0],[0,0,sqrt(2)],[0,0,0]])
    create = annhilate.T
    

    #Generates matrix of zeros
    def zero_mat(N):
        zero_gate = np.array([[0,0,0],[0,0,0],[0,0,0]])
        init = zero_gate
        if N < 2:
            return 1
        for i in range(0,N - 1):
            zero_gate = torch.tensor(np.kron(zero_gate,init))
        return zero_gate
    
    
    #Sums Pauli gates with coefficients (used for X & Y pulses with optimizer coef.)
    def sum_pauli(coef, gate):
        N = len(coef)#number of qubits
        total_pauli = zero_mat(N)
        #Summing all Z gates
        for i in range(0,N):
            pauli_temp = 1
            for j in range(0,i):
                pauli_temp = torch.tensor(np.kron(pauli_temp,id))
            pauli_temp = torch.tensor(np.kron(pauli_temp,gate))
            for j in range(i+1,N):
                pauli_temp = torch.tensor(np.kron(pauli_temp,id))
            total_pauli = total_pauli + coef[i]*pauli_temp
        return total_pauli

    #Variable initializations
    N = len(B)
    torch.manual_seed(9)
    dt = torch.cdouble # datatype and precision
    infidelity_list=torch.zeros([N_iter,1])

    #J coefficients gathering (only if J is in N x N matrix, otherwise set J_coef=J) <- essentially flattens the array
    J_coef = []
    for i in range(0,len(J) - 1):
        for j in range(0,len(J) - i - 1):
            J_coef.append(J[i,j].item())

    #H0 generation, gets all permutations of 2-body coupling 
    permuts = [1,1]
    for i in range(2,N):
        permuts.append(0)
    permuts = list(set(permutations(permuts,N)))
    permuts.sort()
    permuts.reverse()#All permutations of ZZ coupling stored as bit arrays
    H0 = zero_mat(N)

    #Generating Static Hamiltonian
    for i,u in enumerate(permuts):
        H0_temp = 1
        for p in u:
            if p==1:
                H0_temp = torch.tensor(np.kron(H0_temp,(annhilate + create)))
            else:
                H0_temp = torch.tensor(np.kron(H0_temp,id))
        H0 = H0 + J_coef[i]*H0_temp
    print(H0)

    #Generating Energy Levels 
    #H0 = H0 + sum_pauli(B,sz) <- Original from Alexs code. 

    #Unitary group generation (used for fidelity calculation)
    SU = []
    pauli_int = [1, 2, 3, 4, 5, 6, 7, 8, 9]#eq to [sx,sy,sz,id] -> [Gelman Matrix set]
    #pauli_int = [1,2,3,4]
    perms = list(product(pauli_int,repeat=N))#all permutations of paulis
    for p in perms:#mapping integers to pauli 
        unitary = 1
        for pauli in p:
            if pauli == 1:
                unitary = torch.tensor(np.kron(unitary,sx),dtype=torch.cdouble)
            elif pauli == 2:
                unitary = torch.tensor(np.kron(unitary,sy),dtype=torch.cdouble)
            elif pauli == 3:
                unitary = torch.tensor(np.kron(unitary,sz),dtype=torch.cdouble)
            elif pauli == 4:
                unitary = torch.tensor(np.kron(unitary,id),dtype=torch.cdouble)
            elif pauli == 5:
                unitary = torch.tensor(np.kron(unitary,l5),dtype=torch.cdouble)
            elif pauli == 6:
                unitary = torch.tensor(np.kron(unitary,l6),dtype=torch.cdouble)
            elif pauli == 7:
                unitary = torch.tensor(np.kron(unitary,l7),dtype=torch.cdouble)
            elif pauli == 8:
                unitary = torch.tensor(np.kron(unitary,l8),dtype=torch.cdouble)
            elif pauli == 9:
                unitary = torch.tensor(np.kron(unitary,l4),dtype=torch.cdouble)
        SU.append(unitary)

    # MACHINE LEARNING STUFF

    #These are the coefficients we are optimizing
    R = torch.rand([M,2*N], dtype=torch.double) *2*np.pi # Random initialization (between 0 and 2pi)
    R.requires_grad = True # set flag so we can backpropagate
    #Optimizer settings(can be changed & opttimized)
    lr=0.3#learning rate
    opt = 'SGD'  # Choose optimizer - ADAM, SGD (typical). ADAMW, ADAMax, Adadelta,  
                        # Adagrad, Rprop, RMSprop, ASGD, also valid options.     
    sched = 'Plateau'  # Choose learning rate scheduler - Plateau, Exponential (typical), Step
    if opt=='ADAM': optimizer = torch.optim.Adam([R], lr = lr, weight_decay=1e-6)
    elif opt=='ADAMW': optimizer = torch.optim.AdamW([R], lr = lr, weight_decay=0.01)
    elif opt=='ADAMax': optimizer = torch.optim.Adamax([R], lr = lr, weight_decay=0.01)
    elif opt=='RMSprop': optimizer = torch.optim.RMSprop([R], lr = lr, momentum=0.2)
    elif opt=='Rprop': optimizer = torch.optim.Rprop([R], lr = lr)
    elif opt=='Adadelta': optimizer = torch.optim.Adadelta([R], lr = lr) 
    elif opt=='Adagrad': optimizer = torch.optim.Adagrad([R], lr = lr)
    elif opt=='SGD': optimizer = torch.optim.SGD([R], lr = lr, momentum=0.99, nesterov=True)
    elif opt=='ASGD': optimizer = torch.optim.ASGD([R], lr = lr)
    else: optimizer=None; opt='None'
        
    if sched=='Step': scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=N_iter/10, gamma=0.9)
    elif sched=='Exponential': scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    elif sched=='Plateau': scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',min_lr=0.03, factor=0.3 , patience= 20 ); loss_in=True; 
    else: scheduler=None; sched='None'

    # END MACHINE LEARNING STUFF


    #Finding Optimal Pulses
    for n in range(0,N_iter):
        #Creating Drive Hamilontian
        U_Exp = 1
        for i in range(0,N):
            U_Exp = torch.tensor(np.kron(U_Exp,id),dtype=dt)#initializing unitary

        for m in range(0,M):#Product of pulses
            pulse_coef = R[m] 
            H1 = sum_pauli(pulse_coef[:N],raiseOne + raiseOne.T) + sum_pauli(pulse_coef[N:],sqrt(2)*(raiseTwo + raiseTwo.T))
            U_Exp = torch.matmul(torch.matrix_exp(-1j*(H0+H1)*t/M),U_Exp)

        #Fidelity calulcation given by Nielsen Paper
        fidelity = 0
        d = 3**N
        
        for U in SU:
            eps_U = torch.matmul(torch.matmul(U_Exp,U),(U_Exp.conj().T))
            target_U = torch.matmul(torch.matmul(target_gate,(U.conj().T)),(target_gate.conj().T))
            tr = torch.trace(torch.matmul(target_U,eps_U))
            fidelity = fidelity + tr
        fidelity = abs(fidelity + d*d)/(d*d*(d+1))    
        infidelity = 1 - fidelity
        infidelity_list[n] = infidelity.detach()
        infidelity.backward()

        #Printing statement
        if (n+1)%100==0: 
            print('Itertation ', str(n+1), ' out of ', str(N_iter), 'complete. Avg Infidelity: ', str(infidelity.item()))

        #optimizer 
        if optimizer is not None and scheduler is None:  # Update R
            optimizer.step()
            optimizer.zero_grad()
        elif optimizer is not None and scheduler is not None:
            optimizer.step()
            if loss_in: 
                scheduler.step(infidelity)
            else: 
                scheduler.step()
            optimizer.zero_grad()
        else:
            R.data.sub_(lr*R.grad.data) # using data avoids overwriting tensor object
            R.grad.data.zero_()           # and it's respective grad info
    
    print('The infidelity of the generated gate is: ' + str(infidelity_list.min().item()))
    #return R
    return infidelity_list.min().item()

In [112]:
print("Qutrit Sytem reaching 99\% fidelity in pi/4 time for CNOT")
CNOT_qutrit = torch.tensor([[1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],dtype=torch.cdouble)
J = torch.tensor([[1,1],[1,1]])
fidelity_Qutrit(J,[1,1],32,CNOT_qutrit,np.pi,500)

Qutrit Sytem reaching 99\% fidelity in pi/4 time for CNOT
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000],
        [1.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 2.0000],
        [0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 2.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 2.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)
Itertation  100  out of  500 complete. Avg Infidelity:  0.8717111400003361
Itertation  200  out of  500 complete. Avg Infidelity:  0.8509162855771952
Itertation  300  out of  

0.8151859641075134

In [22]:
annhilate = np.array([[0,1,0],[0,0,sqrt(2)],[0,0,0]])
create = annhilate.T

In [20]:
print(create)
print(annhilate)

[[0.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         1.41421356 0.        ]]
[[0.         1.         0.        ]
 [0.         0.         1.41421356]
 [0.         0.         0.        ]]


In [ ]:
def pulse_gen(coef,N1,N2):
    return coef[0]*

In [69]:
def sigma_gen(trit1,trit2,N):
    sigma = np.zeros([9,9])
    np.concatenate([int(t) for t in trit1],[int(t) for t in trit2])
    sigma[int(str(trit2),3),int(str(trit1),3)] = sqrt(max(np.concatenate([int(t) for t in trit1],[int(t) for t in trit2])))
    return sigma


In [70]:
sigma_gen("22","21",2)

TypeError: 'list' object cannot be interpreted as an integer

-10

In [92]:
sigma1 = np.zeros([9,9])
sigma1[0,1] = 1

sigma2 = np.zeros([9,9])
sigma2[1,2] = 1

sigma3 = np.zeros([9,9])
sigma3[0,3] = 1

sigma4 = np.zeros([9,9])
sigma4[3,4] = 1

sigma5 = np.zeros([9,9])
sigma5[4,5] = 1

sigma6 = np.zeros([9,9])
sigma6[3,6] = 1

sigma7 = np.zeros([9,9])
sigma7[6,7] = 1

sigma8 = np.zeros([9,9])
sigma8[7,8] = 1

H = sigma1+sigma2+sigma3+sigma4+sigma5+sigma6+sigma7+sigma8
H = H + H.T
for col in H:
    print(col)

[0. 1. 0. 1. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 1. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 1. 0.]
